# 데이터 수집

## 1. 티어별 랭킹 닉네임 수집 (platinum, emerald, diamond, master, grandmaster, challenger) 1위 - 200위

In [ ]:
import time
import random
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import subprocess
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import json
import os
import urllib.parse

In [ ]:
options = Options()
options.add_argument("--start-maximized")  
options.add_experimental_option("detach", True)

# 크롬 드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

tier_list = ['platinum', 'emerald', 'diamond', 'master', 'grandmaster', 'challenger']
rank_nicknames = []
tiers = []

def get_nicknames(tier, page_number):
    url = f'https://www.op.gg/leaderboards/tier?tier={tier}&page={page_number}'
    headers = {'User-Agent': 'Mozila/5.0'}
    html = requests.get(url, headers=headers)
    time.sleep(2)  

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    elements = soup.find_all('div', class_='tag css-12ijbdy e1swkqyq0')

    for element in elements:
        nickname_text = element.find_all('span')[0].text.strip()
        tag_text = element.find_all('span')[1].text.strip()

        tiers.append(tier)
        rank_nicknames.append(f"{nickname_text} {tag_text}")

for tier in tier_list:
    for page in range(1, 3):
        get_nicknames(tier, page)


df_tier_rank = pd.DataFrame({
    'Tier': tiers,
    'Nickname': rank_nicknames
})

df_tier_rank

In [ ]:
df_tier_rank.to_csv('tier_rank.csv', index=False)

## 2. 유저별 정보 수집 (주챔피언, 게임횟수, 승률, 포지션, 포지션게임횟수)

In [ ]:
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chromeCookie"')

url = 'https://lol.ps'
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
driver.get(url)

# WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.XPATH, '/html/body/main/div[2]/div/div/button[2]'))
# ).click()

data = []
positions = ["탑", "정글", "미드", "원딜", "서폿"]


def get_info(nickname):

    search = nickname
    #driver.get(url)
    #time.sleep(random.uniform(1, 3))  
    time.sleep(1)
    search_f = driver.find_element(By.XPATH, value='//*[@id="default-search"]')
    search_f.clear()
    search_f.send_keys(search)
    if nickname == '모로얌 #KR 1':
        driver.find_element(By.XPATH, value='/html/body/main/div/div[2]/div[1]/div/div[1]/div[3]/button').click()

        
    else:
        driver.find_element(By.XPATH, value='//*[@id="top-menu-container"]/div/div[2]/div[1]/div[1]/div/div[3]/button').click()
   
    
    time.sleep(4) 
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    
    positions = ["탑", "정글", "미드", "원딜", "서폿"]

    numbers = []

    elements = driver.find_elements(By.CSS_SELECTOR, 'div.relative.flex.flex-1.flex-grow.flex-col.items-center.justify-start.gap-2')

    for i, element in enumerate(elements):
        p_elements = element.find_elements(By.TAG_NAME, 'p')
        for p in p_elements:
            text = p.text.strip()
            if text.isdigit(): 
                numbers.append(int(text))  

    max_games = max(numbers)  
    max_position = positions[numbers.index(max_games)]  
        
    champion_data = soup.find('div', class_='flex w-full flex-row justify-between gap-2 px-4 py-2 first:rounded-t last:rounded-b odd:bg-primary-mute even:bg-primary-pale dark:text-dark-psbk dark:odd:bg-dark-primary-mute dark:even:bg-dark-primary-pale desktop:p-2')
    champion_name = champion_data.find('p', class_='text-left text-sm font-bold text-psbk dark:text-dark-psbk').get_text(strip=True)
    black = champion_data.find('p', class_='text-right text-sm font-medium desktop:text-rg desktop:font-bold text-psbk-sub dark:text-dark-psbk-sub')
    
    if black == None : 
        win_rate = champion_data.find('p', class_='text-right text-sm font-medium desktop:text-rg desktop:font-bold text-ui-gn-base dark:text-dark-ui-gn-base').get_text(strip=True)
    else :  
        win_rate = champion_data.find('p', class_='text-right text-sm font-medium desktop:text-rg desktop:font-bold text-psbk-sub dark:text-dark-psbk-sub').get_text(strip=True)

    game_count = champion_data.find('p', class_='text-right text-xs font-medium text-psbk-sub dark:text-dark-psbk-sub').get_text(strip=True)
    
    data.append({
        '닉네임': nickname,
        '주챔피언': champion_name,
        '게임횟수': game_count,
        '승률': win_rate,
        '포지션': max_position,
        '포지션게임횟수': max_games
    })
    
    time.sleep(random.uniform(1, 2))




df_test = df_tier_rank['Nickname'].apply(get_info)
#get_info('BlackGentle #BNG')
df_data = pd.DataFrame(data)
df_combined = df_data.copy

In [ ]:
df_combined['승리횟수'] = df_combined['게임횟수'] * (df_combined['승률'] / 100)
df_combined['승리횟수'] = round(df_combined['승리횟수'])
df_combined = df_combined.drop(columns='승률')
df_combined['승리횟수'] = df_combined['승리횟수'].astype(int)

In [ ]:
new_order = ['Tier', '닉네임', '주챔피언', '게임횟수', '승리횟수', '포지션', '포지션게임횟수']
df_combined = df_combined[new_order]

In [ ]:
df_combined.to_csv('ranker_info.csv', index=False)

## 3. 각 유저별 최근 100개 게임데이터 수집

### 게임데이터 수집 - puuid 수집
- 게임데이터는 각 유저의 고유 번호인 puuid를 수집 후 해당 puuid의 최근 플레이 100개의 플레이 고유번호인 matchID를 가져와야함
- 닉네임, 태그를 사용하여 puuid를 수집 후, puuid로 matchID를 수집
- 그 후 matchID의 게임정보를 불러와서 조합과 승패여부를 최종적으로 수집한다.

In [ ]:
df_combined = pd.read_csv('ranker_info.csv')
df_combined['Tag'] = ''
df_combined['닉네임'].str.strip()

In [ ]:
df_test = df_combined.copy()
df_test['닉네임'] = df_test['닉네임'].str.replace(' ','')
df_test[['닉네임', '태그']] = df_test['닉네임'].str.split('#', expand=True)
df_ID = df_test[['닉네임','태그']]

In [ ]:
API_KEY = 'RGAPI-e967648e-e8af-454a-ae6a-544917e037bb'

Account_Base_URL = 'https://asia.api.riotgames.com/riot/account/v1'

csv_file = "puuid_data.csv"


limit1 = 0
limit2 = 0

def check_limit1():
    global limit1
    limit1 += 1
    if limit1 >= 20:
        print('1초 요청 제한 초과 - 1초 대기')
        limit1 = 0
        time.sleep(1)

def check_limit2():
    global limit2
    limit2 += 1
    if limit2 >= 150:
        print('2분 요청 제한 초과 - 150초 대기')
        limit2 = 0
        time.sleep(120)

def get_puuid(game_name, tag_line):
    url = f'{Account_Base_URL}/accounts/by-riot-id/{game_name}/{tag_line}?api_key={API_KEY}'
    
    while True:  
        response = requests.get(url)
        
        if response.status_code == 200:
            check_limit1()
            check_limit2()
            return response.json().get('puuid')
        
        elif response.status_code == 429:  
            print(f" 요청 초과 (Rate Limit Exceeded) - 150초 대기 후 재시도")
            time.sleep(150) 
        
        else:
            print(f" PUUID 가져오기 실패: {response.status_code}, {game_name}")
            with open("failures.txt", "a", encoding="utf-8") as file:
                file.write(f"{response.status_code}, {game_name}\n")
            return None  

if os.path.exists(csv_file):
    df_existing = pd.read_csv(csv_file)
    processed_names = set(df_existing["닉네임"].tolist())  
else:
    df_existing = pd.DataFrame()
    processed_names = set()


df_puuid_list = []


for index, row in df_ID.iterrows():
    game_name = row['닉네임']
    tag_line = row['태그']

    if game_name in processed_names:
        print(f"{game_name} - 이미 처리됨, 건너뜀")
        continue  

    print(f"{game_name} - PUUID 가져오는 중")
    puuid = get_puuid(game_name, tag_line)

    if puuid:  
        puuid_info = {
            "닉네임": game_name,
            "태그": tag_line,
            "PUUID": puuid
        }

        df_puuid_list.append(puuid_info)

        df_temp = pd.DataFrame([puuid_info])
        df_temp.to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False)

df_final = pd.concat([df_existing, pd.DataFrame(df_puuid_list)], ignore_index=True)

df_final.to_csv(csv_file, index=False)  

In [ ]:
df_ID

,닉네임,태그,puuid
0,초현식근동이물,KR1,q_u5DWr6hDSszpqvVVn1d_imwlpNOxKckgy9OB6GvZ6Md5...
1,매우라,5838,sjXRf54RrDHeMw4xW_sudjnFO2R2cLzpQZkwzKZzuRycvj...
2,Dancin,kr2,BVGcyx8KDhcpwm8lD47MOJF3jZka7jCHVvfJRPUpRGfgHK...
3,Namy000n,5111,GCUsWx3eaSYUhCG_glmDzaMREWD2GuCIWiQG_j2jVVwgLV...
4,Fly over you,wing,DvsV2U6duv15hx1tQCagd2WeQ6rjxx8FWL-z-ql_512PJG...
...,...,...,...
1194,Chu성훈,KR1,upxh9FIC3JQuJJhRSMYnRvb0cD9ADC_mAtS8U_HDI__V0i...
1195,The Herta,Hokai,LBIyQAg4PLalKMnNRbQm3EjhQ3tASMCj9I1K5oRPvlKLHf...
1196,렝 화,렝 화,3ZKSU0Ygv_28kfL8tVrx-MrOlOFO8CzAQ7sP9_eWdk5E18...
1197,kfo,KR1,oaV9CwPRTk-CMd-j7NoHGKBlOb2XoU91Kmr1T1B2wnDGQS...


- 누락확인

In [ ]:
df_ID[df_ID['puuid'].isna()]

,닉네임,태그,puuid
331,FADE,ZED,NaN
523,Bugwon,부구언,NaN
1057,빨뚜참이슬,하이트,NaN


- 닉네임 변경 등으로 puuid에 누락이 있다.
- 인덱스 구성 : <br>
0 ~ 199 : platinum <br>
200 ~ 399 : emerald <br>
400 ~ 599 : diamond <br>
600 ~ 799 : master <br>
800 ~ 999 : grandmaster <br>
1000 ~ 1999 : challenger <br>

- 누락이 있는 인덱스 : 331, 523, 1057로 각각 emerald, diamond, challenger에서 누락이 있다.
- 랭킹은 매일 매시간 변화하므로 현재 랭킹과 비교한 후 고유값으로 대체

In [ ]:
df_tier_rank_0205 = pd.read_csv('tier_rank_0205.csv')
eme = df_tier_rank_0205[df_tier_rank_0205['Tier'] == 'emerald']
dia = df_tier_rank_0205[df_tier_rank_0205['Tier'] == 'diamond']
chall = df_tier_rank_0205[df_tier_rank_0205['Tier'] == 'challenger']

In [ ]:
options = Options()
options.add_argument("--start-maximized")  
options.add_experimental_option("detach", True)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

tier_list = ['emerald', 'diamond', 'challenger']
rank_nicknames = []
tiers = []

def get_nicknames(tier, page_number):
    url = f'https://www.op.gg/leaderboards/tier?tier={tier}&page={page_number}'
    driver.get(url)
    time.sleep(2) 

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    elements = soup.find_all('div', class_='tag css-12ijbdy e1swkqyq0')

    for element in elements:
        nickname_text = element.find_all('span')[0].text.strip()
        tag_text = element.find_all('span')[1].text.strip()

        tiers.append(tier)
        rank_nicknames.append(f"{nickname_text} {tag_text}")

for tier in tier_list:
    for page in range(1, 3):  
        get_nicknames(tier, page)

df_tier_rank = pd.DataFrame({
    'Tier': tiers,
    'Nickname': rank_nicknames
})

df_tier_rank

,Tier,Nickname
0,emerald,팬이에요 #KR2
1,emerald,ColorLife #123
2,emerald,RYAN #리안06
3,emerald,스웨인의신스웨인의왕스웨인그자체 #모기맨
4,emerald,전화찬스 #KR1
...,...,...
595,challenger,시다림 #V H
596,challenger,The Herta #Hokai
597,challenger,Sungod #KR2
598,challenger,안수은 #1218


In [ ]:
emerald = df_tier_rank[df_tier_rank['Tier'] == 'emerald']
diamond = df_tier_rank[df_tier_rank['Tier'] == 'diamond']
challenger = df_tier_rank[df_tier_rank['Tier'] == 'challenger']

In [ ]:
df_E_merged = pd.concat([eme, emerald], ignore_index=True)
df_D_merged = pd.concat([dia, diamond], ignore_index=True)
df_C_merged = pd.concat([chall, challenger], ignore_index=True)

- 병합된 데이터프레임에서 중복 닉네임 제거

In [ ]:
df_E_merged.drop_duplicates(subset=['Nickname'], keep=False)

,Tier,Nickname
0,emerald,n bee #1998
4,emerald,의문의 귤까기고수 #KR2
6,emerald,magum #KR2
10,emerald,GAME BOOM #KR1
13,emerald,머리는차갑게맘은뜨겁게 #KR2
...,...,...
392,emerald,바다와 도롱뇽 #켄시요네즈
394,emerald,짱 l 아 #KR1
396,emerald,Referee #KR3
397,emerald,空への飛び方 #1119


In [ ]:
df_D_merged.drop_duplicates(subset=['Nickname'], keep=False)

,Tier,Nickname
0,diamond,Legacy #KR2
6,diamond,마트료시카입니다 #KR1
7,diamond,오픈해본적없음 #KR0
9,diamond,rnjswlscks #KR1
10,diamond,살려쥬시쿨 #KR1
...,...,...
395,diamond,응애나 #KR1
396,diamond,mflkxzcklfj #KR1
397,diamond,devil galio #hmbb
398,diamond,알렉산더 동 #KR1


In [ ]:
df_D_merged.drop_duplicates(subset=['Nickname'], keep=False)

,Tier,Nickname
0,diamond,Legacy #KR2
6,diamond,마트료시카입니다 #KR1
7,diamond,오픈해본적없음 #KR0
9,diamond,rnjswlscks #KR1
10,diamond,살려쥬시쿨 #KR1
...,...,...
395,diamond,응애나 #KR1
396,diamond,mflkxzcklfj #KR1
397,diamond,devil galio #hmbb
398,diamond,알렉산더 동 #KR1


In [ ]:
df_ID.loc[331] = ['바다와 도롱뇽', '켄시요네즈','0Q_mp0r_oenAeZhmuFR5D-hU9A6KgQCEe0_x_ybJbTNyyNLq9DMVpNE8m5PU-HKFFJ8yOxFjWgK5wQ']
df_ID.loc[523] = ['응애나', 'KR1', 'kCaWiR-9RthXPAW4hf25WUBCzFEutLEiu_tkATWx4_di6C-kOMfpIJFx1_vkj-KQm6yUVMhvjRUbPg']
df_ID.loc[1057] = ['Coldplay', '000', '9F8PaCv9N_5xHv2oAtOmQWWH8TmhXH9L0UhUPriGeqjke_pATv0TOYe0TENe5zgKeZUMf4dOONwxgw']

In [ ]:
df_ID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     1199 non-null   object
 1   태그      1199 non-null   object
 2   puuid   1199 non-null   object
dtypes: object(3)
memory usage: 28.2+ KB


- 누락처리 완료

In [ ]:
df_ID.to_csv('puuid_final.csv', index=False)

### 게임데이터 수집 - matchID 수집

In [ ]:
df_user_info = pd.read_csv('puuid_final.csv')

In [ ]:
api_key = 'RGAPI-73f5e044-4592-4272-9f58-d7cbd142ace6'

base_url = "https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=100"

csv_file = "matchid_0206.csv"

limit1 = 0
limit2 = 0

def check_limit1():
    global limit1
    limit1 += 1
    if limit1 >= 20:
        print('1초 요청 제한 초과 - 1초 대기')
        limit1 = 0
        time.sleep(1)

def check_limit2():
    global limit2
    limit2 += 1
    
    if limit2 >= 100:
        print('2분 요청 제한 초과 - 150초 대기')
        limit2 = 0
        time.sleep(150)

def get_match(puuid):
    url = base_url.format(puuid=urllib.parse.quote(puuid))

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "Accept-Language": "ko,en;q=0.9,en-US;q=0.8",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }

    while True: 
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            check_limit1()
            check_limit2()
            return response.json()  
        
        elif response.status_code == 429: 
            print(f" 요청 초과 (Rate Limit Exceeded) - 150초 대기 후 재시도")
            time.sleep(150)  

        else:
            print(f" API 요청 오류: {response.status_code}, {response.text}")
            with open("failures.txt", "a", encoding="utf-8") as file:
                file.write(f"{response.status_code}, {puuid}\n")
            return None  

if os.path.exists(csv_file):
    df_existing = pd.read_csv(csv_file)
    processed_puuids = set(df_existing["puuid"].tolist())  
else:
    df_existing = pd.DataFrame()
    processed_puuids = set()

df_match_list = []

for puuid in df_user_info['PUUID']:
    if puuid in processed_puuids:
        print(f"{puuid} - 이미 처리됨, 건너뜀")
        continue 

    print(f"{puuid} - matchID 가져오는 중")
    match_ids = get_match(puuid)

    if match_ids:  
        temp_df = pd.DataFrame({"puuid": [puuid] * len(match_ids), "matchID": match_ids})
        df_match_list.append(temp_df)

        temp_df.to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False)

df_final = pd.concat([df_existing] + df_match_list, ignore_index=True)

df_final.to_csv(csv_file, index=False)  


### 게임데이터 수집 - 팀별 조합과 승패여부 수집

In [ ]:
df_match_id = pd.read_csv('matchid.csv')

In [ ]:
csv_file = "match_data_progress_최종.csv"

api_key = 'RGAPI-16db5d1c-e78b-4b8a-863a-9455e45601c0'

base_url = "https://asia.api.riotgames.com/lol/match/v5/matches/{matchid}"


limit1 = 0  
limit2 = 0  

def check_limit1():
    global limit1
    limit1 += 1
    if limit1 >= 20:
        print('1초 요청 제한 초과 - 1초 대기')
        limit1 = 0
        time.sleep(1)

def check_limit2():
    global limit2
    limit2 += 1
    if limit2 >= 100:
        print('2분 요청 제한 초과 - 150초 대기')
        limit2 = 0
        time.sleep(150)  


def get_compose(matchid):
    url = base_url.format(matchid=urllib.parse.quote(matchid))

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "Accept-Language": "ko,en;q=0.9,en-US;q=0.8",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }

    while True:  
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            check_limit1()
            check_limit2()
            return response.json()  

        elif response.status_code == 429:  
            print(f" 요청 초과 (Rate Limit Exceeded) - 150초 대기 후 재시도")
            time.sleep(150) 

        else:
            print(f" API 요청 오류: {response.status_code}, {response.text}")
            with open("compose_failures.txt", "a", encoding="utf-8") as file : 
                file.write(f"{response.status_code}, {matchid}\n")
            return None  


match_ids = df_match_id['matchID'].tolist()  


if os.path.exists(csv_file):
    df_existing = pd.read_csv(csv_file)
    processed_match_ids = set(df_existing["matchID"].tolist())  
else:
    df_existing = pd.DataFrame()
    processed_match_ids = set()

df_matches = []

for match_id in match_ids:
    if match_id in processed_match_ids:
        print(f"{match_id} - 이미 처리됨, 건너뜀")
        continue  

    print(f"{match_id} - 데이터 가져오는 중")
    match_data = get_compose(match_id)
   
    if match_data:  
        teams = {100: {}, 200: {}}

    
        for p in match_data["info"]["participants"]:
            team = p["teamId"]
            lane = p["teamPosition"]  
            champ_id = p["championId"]
            win = p["win"]  

            teams[team][lane] = champ_id

        winning_team = "팀1" if match_data["info"]["teams"][0]["win"] else "팀2"

        match_info = {
            "matchID": match_id,
            "팀1_TOP": teams[100].get("TOP", None),
            "팀1_JUNGLE": teams[100].get("JUNGLE", None),
            "팀1_MID": teams[100].get("MIDDLE", None),  
            "팀1_BOTTOM": teams[100].get("BOTTOM", None),
            "팀1_SUPPORT": teams[100].get("UTILITY", None),  
            "팀2_TOP": teams[200].get("TOP", None),
            "팀2_JUNGLE": teams[200].get("JUNGLE", None),
            "팀2_MID": teams[200].get("MIDDLE", None),
            "팀2_BOTTOM": teams[200].get("BOTTOM", None),
            "팀2_SUPPORT": teams[200].get("UTILITY", None),
            "승패": winning_team
        }

        df_matches.append(match_info)

        df_temp = pd.DataFrame([match_info])
        df_temp.to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False)

df_final = pd.concat([df_existing, pd.DataFrame(df_matches)], ignore_index=True)

df_final.to_csv(csv_file, index=False)  


데이터프레임 정제

In [ ]:
combination = pd.read_csv('match_data_progress_최종.csv')

In [ ]:
combination = combination.dropna()
combination

,matchID,팀1_TOP,팀1_JUNGLE,팀1_MID,팀1_BOTTOM,팀1_SUPPORT,팀2_TOP,팀2_JUNGLE,팀2_MID,팀2_BOTTOM,팀2_SUPPORT,승패
0,KR_7510067698,799.0,245.0,238.0,81.0,555.0,58.0,121.0,39.0,51.0,25.0,팀1
1,KR_7509984768,897.0,131.0,238.0,81.0,33.0,266.0,80.0,112.0,145.0,412.0,팀2
2,KR_7508526757,14.0,121.0,112.0,42.0,117.0,266.0,876.0,777.0,81.0,43.0,팀2
3,KR_7508455937,82.0,254.0,7.0,81.0,526.0,92.0,59.0,268.0,202.0,43.0,팀2
4,KR_7508410439,8.0,56.0,910.0,429.0,53.0,92.0,28.0,238.0,51.0,43.0,팀1
...,...,...,...,...,...,...,...,...,...,...,...,...
85663,KR_7472692827,126.0,76.0,91.0,21.0,43.0,799.0,64.0,13.0,42.0,89.0,팀2
85664,KR_7472262484,58.0,113.0,112.0,110.0,201.0,80.0,72.0,268.0,21.0,526.0,팀1
85665,KR_7472204953,887.0,234.0,142.0,42.0,497.0,897.0,154.0,61.0,21.0,201.0,팀2
85666,KR_7472139354,799.0,60.0,39.0,81.0,201.0,126.0,76.0,246.0,145.0,526.0,팀1


In [ ]:
df_team1 = combination[["matchID", "팀1_TOP", "팀1_JUNGLE", "팀1_MID", "팀1_BOTTOM", "팀1_SUPPORT"]].copy()
df_team1.columns = ["matchID", "TOP", "JUNGLE", "MID", "BOTTOM", "SUPPORT"]
df_team1["승패"] = (combination["승패"] == "팀1").astype(int)

df_team2 = combination[["matchID", "팀2_TOP", "팀2_JUNGLE", "팀2_MID", "팀2_BOTTOM", "팀2_SUPPORT"]].copy()
df_team2.columns = ["matchID", "TOP", "JUNGLE", "MID", "BOTTOM", "SUPPORT"]
df_team2["승패"] = (combination["승패"] == "팀2").astype(int)


final_df = pd.concat([df_team1, df_team2], ignore_index=True)

grouped_df = final_df.groupby("matchID").apply(lambda x: x.reset_index(drop=True))


In [ ]:
team_com = grouped_df.reset_index(drop=True)

In [ ]:
team_com['TOP'] = team_com['TOP'].astype(int)
team_com['JUNGLE'] = team_com['JUNGLE'].astype(int)
team_com['MID'] = team_com['MID'].astype(int)
team_com['BOTTOM'] = team_com['BOTTOM'].astype(int)
team_com['SUPPORT'] = team_com['SUPPORT'].astype(int)

In [ ]:
json_file_path = "champion.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    champion_data = json.load(f)

champion_mapping = {champ_info["key"]: champ_info["name"] for champ_info in champion_data["data"].values()}


for role in ["TOP", "JUNGLE", "MID", "BOTTOM", "SUPPORT"]:
    team_com[role] = team_com[role].astype(str).map(champion_mapping)

team_com

,matchID,TOP,JUNGLE,MID,BOTTOM,SUPPORT,승패
0,KR_6447345269,럼블,헤카림,사일러스,자야,쓰레쉬,0
1,KR_6447345269,렝가,리 신,아리,징크스,모르가나,1
2,KR_6447377297,세주아니,리 신,알리스타,드레이븐,룰루,1
3,KR_6447377297,잭스,럼블,제드,자야,모르가나,0
4,KR_6447414057,탐 켄치,케인,아리,야스오,신드라,0
...,...,...,...,...,...,...,...
169755,KR_7515577731,멜,케인,미스 포츈,루시안,유미,0
169756,KR_7515578129,카르마,벨베스,탈리야,이즈리얼,조이,1
169757,KR_7515578129,크산테,킨드레드,카시오페아,드레이븐,룰루,0
169758,KR_7515583365,이렐리아,다이애나,빅토르,트리스타나,블리츠크랭크,1


In [ ]:
team_com.to_csv('match_combination.csv', index=False)

## 4. 티어별 챔피언 랭킹 정보 (라인, 승률, 픽률, 밴률, 게임횟수)

- 플래티넘 티어 챔피언 랭킹

In [ ]:
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chromeCookie"')

url = 'https://lol.ps'
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
driver.get(url)


driver.find_element(By.XPATH, value='//*[@id="menu-container"]/div/div[2]/div[1]/a').click()
time.sleep(4)
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[2]/div[2]/button').click()
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[2]/div[2]/ul/li[4]').click()
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[1]/div[6]').click()
time.sleep(4)

scrollable_div = driver.find_element(By.CSS_SELECTOR, "div.dark\\:border-y-dark-psgr-formtext.desktop\\:max-h-\\[600px\\].desktop\\:overflow-y-scroll.desktop\\:border-y")

scroll_pause_time = 2  

last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

while True:
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 300;", scrollable_div)
    time.sleep(scroll_pause_time)
    
    new_height = driver.execute_script("return arguments[0].scrollTop", scrollable_div)
    
    if new_height == last_height:
        break  
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

content_container = soup.find('div', id='content-container', class_='mx-auto min-w-[280px] bg-white dark:bg-dark-psgr-darkest desktop:mt-4 desktop:w-[1080px] desktop:min-w-[1080px] desktop:rounded-xl desktop:p-8')
tierlist_section = content_container.find('section', id='section_tierlist', class_='mb-14 w-full max-w-[1016px] text-psbk dark:text-dark-psbk desktop:w-full desktop:min-w-[1016px]')
data_container = tierlist_section.find('div', class_='h-full w-full text-psbk dark:text-dark-psbk')
scrollable_container = data_container.find('div', class_='dark:border-y-dark-psgr-formtext desktop:max-h-[600px] desktop:overflow-y-scroll desktop:border-y')


all_rows = scrollable_container.find_all('a')

rows = [
    row for row in all_rows 
    if any(tier in ' '.join(row.get('class', [])) for tier in ['bg-tier-one-hover', 'bg-tier-two-hover', 'bg-tier-three-hover', 'bg-tier-four-hover', 'bg-tier-five-hover'])
]

P_rank = []

for row in rows:
    rank = row.find('div', class_='hidden w-8 flex-none items-center justify-center text-center text-md font-bold text-psbk dark:text-dark-psbk desktop:flex').text.strip()
    champion_name = row.find('span', class_='desktop: absolute left-24 top-0 inline-flex h-4 w-[200px] flex-grow items-center text-sm font-bold text-psbk-sub dark:text-dark-psbk-sub mobile-small:left-[100px] desktop:static desktop:w-auto desktop:pl-2 desktop:text-rg desktop:text-psbk dark:desktop:text-dark-psbk').text.strip()
    win_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[0].text.strip()
    pick_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[1].text.strip()
    ban_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[2].text.strip()
    play_count = row.find_all('div', class_='hidden h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:flex desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[1].text.strip()
    lane = row.find('div', class_='hidden w-24 flex-none items-center text-center desktop:flex').find('span').text.strip()
    
    P_rank.append({
        'Rank': rank,
        'Champion Name': champion_name,
        'Lane' : lane,
        'Win Rate': win_rate,
        'Pick Rate': pick_rate,
        'Ban Rate': ban_rate,
        'Play Count': play_count
    })


df_P_rank = pd.DataFrame(P_rank)
df_P_rank

- 에메랄드 티어 챔피언 랭킹

In [ ]:
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chromeCookie"')

url = 'https://lol.ps'
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
driver.get(url)


driver.find_element(By.XPATH, value='//*[@id="menu-container"]/div/div[2]/div[1]/a').click()
time.sleep(4)
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[2]/div[2]/button').click()
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[2]/div[2]/ul/li[1]').click()
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[1]/div[6]').click()
time.sleep(4)

scrollable_div = driver.find_element(By.CSS_SELECTOR, "div.dark\\:border-y-dark-psgr-formtext.desktop\\:max-h-\\[600px\\].desktop\\:overflow-y-scroll.desktop\\:border-y")

scroll_pause_time = 2  

last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

while True:
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 300;", scrollable_div)
    time.sleep(scroll_pause_time)
    
    new_height = driver.execute_script("return arguments[0].scrollTop", scrollable_div)
    
    if new_height == last_height:
        break  
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


content_container = soup.find('div', id='content-container', class_='mx-auto min-w-[280px] bg-white dark:bg-dark-psgr-darkest desktop:mt-4 desktop:w-[1080px] desktop:min-w-[1080px] desktop:rounded-xl desktop:p-8')
tierlist_section = content_container.find('section', id='section_tierlist', class_='mb-14 w-full max-w-[1016px] text-psbk dark:text-dark-psbk desktop:w-full desktop:min-w-[1016px]')
data_container = tierlist_section.find('div', class_='h-full w-full text-psbk dark:text-dark-psbk')
scrollable_container = data_container.find('div', class_='dark:border-y-dark-psgr-formtext desktop:max-h-[600px] desktop:overflow-y-scroll desktop:border-y')


all_rows = scrollable_container.find_all('a')

rows = [
    row for row in all_rows 
    if any(tier in ' '.join(row.get('class', [])) for tier in ['bg-tier-one-hover', 'bg-tier-two-hover', 'bg-tier-three-hover', 'bg-tier-four-hover', 'bg-tier-five-hover'])
]


E_rank = []

for row in rows:

    rank = row.find('div', class_='hidden w-8 flex-none items-center justify-center text-center text-md font-bold text-psbk dark:text-dark-psbk desktop:flex').text.strip()
    champion_name = row.find('span', class_='desktop: absolute left-24 top-0 inline-flex h-4 w-[200px] flex-grow items-center text-sm font-bold text-psbk-sub dark:text-dark-psbk-sub mobile-small:left-[100px] desktop:static desktop:w-auto desktop:pl-2 desktop:text-rg desktop:text-psbk dark:desktop:text-dark-psbk').text.strip()
    win_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[0].text.strip()
    pick_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[1].text.strip()
    ban_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[2].text.strip()
    play_count = row.find_all('div', class_='hidden h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:flex desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[1].text.strip()
    lane = row.find('div', class_='hidden w-24 flex-none items-center text-center desktop:flex').find('span').text.strip()
    
    E_rank.append({
        'Rank': rank,
        'Champion Name': champion_name,
        'Lane' : lane,
        'Win Rate': win_rate,
        'Pick Rate': pick_rate,
        'Ban Rate': ban_rate,
        'Play Count': play_count
    })


df_E_rank = pd.DataFrame(E_rank)
df_E_rank


- 다이아 티어 챔피언 랭킹

In [ ]:
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chromeCookie"')

url = 'https://lol.ps'
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
driver.get(url)


driver.find_element(By.XPATH, value='//*[@id="menu-container"]/div/div[2]/div[1]/a').click()
time.sleep(4)
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[2]/div[2]/button').click()
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[2]/div[2]/ul/li[2]').click()
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[1]/div[6]').click()
time.sleep(4)

scrollable_div = driver.find_element(By.CSS_SELECTOR, "div.dark\\:border-y-dark-psgr-formtext.desktop\\:max-h-\\[600px\\].desktop\\:overflow-y-scroll.desktop\\:border-y")

scroll_pause_time = 2  

last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

while True:
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 300;", scrollable_div)
    time.sleep(scroll_pause_time)
    
    new_height = driver.execute_script("return arguments[0].scrollTop", scrollable_div)
    
    if new_height == last_height:
        break 
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


content_container = soup.find('div', id='content-container', class_='mx-auto min-w-[280px] bg-white dark:bg-dark-psgr-darkest desktop:mt-4 desktop:w-[1080px] desktop:min-w-[1080px] desktop:rounded-xl desktop:p-8')
tierlist_section = content_container.find('section', id='section_tierlist', class_='mb-14 w-full max-w-[1016px] text-psbk dark:text-dark-psbk desktop:w-full desktop:min-w-[1016px]')
data_container = tierlist_section.find('div', class_='h-full w-full text-psbk dark:text-dark-psbk')
scrollable_container = data_container.find('div', class_='dark:border-y-dark-psgr-formtext desktop:max-h-[600px] desktop:overflow-y-scroll desktop:border-y')


all_rows = scrollable_container.find_all('a')

rows = [
    row for row in all_rows 
    if any(tier in ' '.join(row.get('class', [])) for tier in ['bg-tier-one-hover', 'bg-tier-two-hover', 'bg-tier-three-hover', 'bg-tier-four-hover', 'bg-tier-five-hover'])
]

D_rank = []

for row in rows:

    rank = row.find('div', class_='hidden w-8 flex-none items-center justify-center text-center text-md font-bold text-psbk dark:text-dark-psbk desktop:flex').text.strip()
    champion_name = row.find('span', class_='desktop: absolute left-24 top-0 inline-flex h-4 w-[200px] flex-grow items-center text-sm font-bold text-psbk-sub dark:text-dark-psbk-sub mobile-small:left-[100px] desktop:static desktop:w-auto desktop:pl-2 desktop:text-rg desktop:text-psbk dark:desktop:text-dark-psbk').text.strip()
    win_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[0].text.strip()
    pick_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[1].text.strip()
    ban_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[2].text.strip()
    play_count = row.find_all('div', class_='hidden h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:flex desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[1].text.strip()
    lane = row.find('div', class_='hidden w-24 flex-none items-center text-center desktop:flex').find('span').text.strip()

    D_rank.append({
        'Rank': rank,
        'Champion Name': champion_name,
        'Lane' : lane,
        'Win Rate': win_rate,
        'Pick Rate': pick_rate,
        'Ban Rate': ban_rate,
        'Play Count': play_count
    })


df_D_rank = pd.DataFrame(D_rank)
df_D_rank


- 마스터 티어 챔피언 랭킹

In [ ]:
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chromeCookie"')

url = 'https://lol.ps'
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
driver.get(url)

driver.find_element(By.XPATH, value='//*[@id="menu-container"]/div/div[2]/div[1]/a').click()
time.sleep(4)
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[2]/div[2]/button').click()
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[2]/div[2]/ul/li[3]').click()
driver.find_element(By.XPATH, value='//*[@id="section_tierlist"]/div[2]/div[1]/div[6]').click()
time.sleep(4)

scrollable_div = driver.find_element(By.CSS_SELECTOR, "div.dark\\:border-y-dark-psgr-formtext.desktop\\:max-h-\\[600px\\].desktop\\:overflow-y-scroll.desktop\\:border-y")

scroll_pause_time = 2  

last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

while True:
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 300;", scrollable_div)
    time.sleep(scroll_pause_time)
    

    new_height = driver.execute_script("return arguments[0].scrollTop", scrollable_div)
    
    if new_height == last_height:
        break  
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


content_container = soup.find('div', id='content-container', class_='mx-auto min-w-[280px] bg-white dark:bg-dark-psgr-darkest desktop:mt-4 desktop:w-[1080px] desktop:min-w-[1080px] desktop:rounded-xl desktop:p-8')
tierlist_section = content_container.find('section', id='section_tierlist', class_='mb-14 w-full max-w-[1016px] text-psbk dark:text-dark-psbk desktop:w-full desktop:min-w-[1016px]')
data_container = tierlist_section.find('div', class_='h-full w-full text-psbk dark:text-dark-psbk')
scrollable_container = data_container.find('div', class_='dark:border-y-dark-psgr-formtext desktop:max-h-[600px] desktop:overflow-y-scroll desktop:border-y')


all_rows = scrollable_container.find_all('a')

rows = [
    row for row in all_rows 
    if any(tier in ' '.join(row.get('class', [])) for tier in ['bg-tier-one-hover', 'bg-tier-two-hover', 'bg-tier-three-hover', 'bg-tier-four-hover', 'bg-tier-five-hover'])
]

M_rank = []

for row in rows:
    rank = row.find('div', class_='hidden w-8 flex-none items-center justify-center text-center text-md font-bold text-psbk dark:text-dark-psbk desktop:flex').text.strip()
    champion_name = row.find('span', class_='desktop: absolute left-24 top-0 inline-flex h-4 w-[200px] flex-grow items-center text-sm font-bold text-psbk-sub dark:text-dark-psbk-sub mobile-small:left-[100px] desktop:static desktop:w-auto desktop:pl-2 desktop:text-rg desktop:text-psbk dark:desktop:text-dark-psbk').text.strip()
    win_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[0].text.strip()
    pick_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[1].text.strip()
    ban_rate = row.find_all('div', class_='flex h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[2].text.strip()
    play_count = row.find_all('div', class_='hidden h-14 flex-none items-end justify-end self-end border-l border-psgr-formborder px-1 pb-2 text-sm dark:border-dark-psgr-border mobile-small:text-rg mobile-large:min-w-[72px] desktop:flex desktop:items-center desktop:self-center desktop:pb-0 desktop:pr-2 desktop:text-md desktop:font-medium')[1].text.strip()
    lane = row.find('div', class_='hidden w-24 flex-none items-center text-center desktop:flex').find('span').text.strip()

    M_rank.append({
        'Rank': rank,
        'Champion Name': champion_name,
        'Lane' : lane,
        'Win Rate': win_rate,
        'Pick Rate': pick_rate,
        'Ban Rate': ban_rate,
        'Play Count': play_count
    })


df_M_rank = pd.DataFrame(M_rank)
df_M_rank

In [ ]:
new_col = ['Tier', 'Rank', 'Champion Name', 'Lane', 'Win Rate', 'Pick Rate', 'Ban Rate',
       'Play Count']


df_E_rank['Tier'] = 'Emerald'
df_D_rank['Tier'] = 'Diamond'
df_M_rank['Tier'] = 'Master'
df_P_rank = df_P_rank[new_col]
df_E_rank = df_E_rank[new_col]
df_D_rank = df_D_rank[new_col]
df_M_rank = df_M_rank[new_col]

- 하나의 데이터프레임으로 합치기

In [ ]:
df_champ_combined = pd.concat([df_P_rank, df_E_rank, df_D_rank, df_M_rank], ignore_index=True)
df_champ_combined

,Tier,Rank,Champion Name,Lane,Win Rate,Pick Rate,Ban Rate,Play Count
0,Platinum,1,미스 포츈,바텀,53.99%,23.65%,47.80%,"495,304"
1,Platinum,2,녹턴,정글,52.63%,10.94%,19.01%,"229,182"
2,Platinum,3,리 신,정글,49.74%,26.41%,35.41%,"553,026"
3,Platinum,4,말자하,미드,52.93%,5.27%,4.71%,"110,277"
4,Platinum,5,비에고,정글,50.60%,17.17%,14.48%,"359,595"
...,...,...,...,...,...,...,...,...
969,Master,230,스몰더,바텀,43.61%,1.85%,0.26%,743
970,Master,231,말파이트,탑,42.86%,1.69%,1.37%,679
971,Master,232,스몰더,미드,43.65%,0.76%,0.26%,307
972,Master,233,제라스,서폿,42.40%,2.03%,1.60%,816


In [ ]:
df_champ_combined.to_csv('champ_list.csv', index=False)